In [1]:
import pandas as pd

data = pd.read_csv('connectionlossdata.csv',low_memory=False)
data.head()

,Unnamed: 0,TOPLAMSAAT,SORUNSAYISINET,PortErrorCount,LostCarrierCount,GecenAyPortErrorSayisi,GecmisFark,Kopma,Atlak,POSSEHIR,POSSEMT,SANTRALADI,GeneralDate,anonimid
0,0,22,1,0,0,0,0,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-1-54,2020-12-01,14216756
1,1,22,1,0,0,0,0,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-İSTASYON-54,2020-12-01,14216756
2,2,46,3,1,0,0,1,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-1-54,2020-12-02,14216756
3,3,46,3,1,0,0,1,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-İSTASYON-54,2020-12-02,14216756
4,4,50,3,1,0,0,1,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-1-54,2020-12-03,14216756


In [15]:
unique_cat = len(data['Kopma'].unique())
print("unique category '{cat}'".format(cat=unique_cat))
print(data['Kopma'].unique())

unique category '3'
[nan 'Sık Sık Kopuyor' 'İnternete Erişemiyor']


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4835379 entries, 0 to 4835378
Data columns (total 14 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   Unnamed: 0              int64 
 1   TOPLAMSAAT              int64 
 2   SORUNSAYISINET          int64 
 3   PortErrorCount          int64 
 4   LostCarrierCount        int64 
 5   GecenAyPortErrorSayisi  int64 
 6   GecmisFark              int64 
 7   Kopma                   object
 8   Atlak                   object
 9   POSSEHIR                object
 10  POSSEMT                 object
 11  SANTRALADI              object
 12  GeneralDate             object
 13  anonimid                int64 
dtypes: int64(8), object(6)
memory usage: 516.5+ MB


In [3]:
# decide which categorical variables you want to use in model
for col_name in data.columns:
    type_name = data[col_name].dtypes
    unique_cat = len(data[col_name].unique())
    print("Feature '{col_name}'('{type_name}') has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat,type_name=type_name))

Feature 'Unnamed: 0'('int64') has 4835379 unique categories
Feature 'TOPLAMSAAT'('int64') has 345 unique categories
Feature 'SORUNSAYISINET'('int64') has 795 unique categories
Feature 'PortErrorCount'('int64') has 335 unique categories
Feature 'LostCarrierCount'('int64') has 475 unique categories
Feature 'GecenAyPortErrorSayisi'('int64') has 49 unique categories
Feature 'GecmisFark'('int64') has 337 unique categories
Feature 'Kopma'('object') has 3 unique categories
Feature 'Atlak'('object') has 2 unique categories
Feature 'POSSEHIR'('object') has 212 unique categories
Feature 'POSSEMT'('object') has 7550 unique categories
Feature 'SANTRALADI'('object') has 5451 unique categories
Feature 'GeneralDate'('object') has 13 unique categories
Feature 'anonimid'('int64') has 327436 unique categories


In [6]:
# we can remove the unnecessary features.
data = data.drop('Unnamed: 0',1)
data = data.drop('anonimid',1)

# Also 'POSSEMT', 'SANTRALADI' features has too many unique categories. We can remove them.
data = data.drop('POSSEMT',1)
data = data.drop('SANTRALADI',1)

KeyError: "['Unnamed: 0'] not found in axis"

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4835379 entries, 0 to 4835378
Data columns (total 10 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   TOPLAMSAAT              int64 
 1   SORUNSAYISINET          int64 
 2   PortErrorCount          int64 
 3   LostCarrierCount        int64 
 4   GecenAyPortErrorSayisi  int64 
 5   GecmisFark              int64 
 6   Kopma                   object
 7   Atlak                   object
 8   POSSEHIR                object
 9   GeneralDate             object
dtypes: int64(6), object(4)
memory usage: 368.9+ MB


In [9]:
data['Atlak'].value_counts().sort_values(ascending=False).head(10)

Atlak Bağlantı Sorunu    2
Name: Atlak, dtype: int64

In [28]:
# datadaki illerin dağılımınada bakalım, istanbul, ankara ve izmir domine ediyor.
data['POSSEHIR'].value_counts().sort_values(ascending=False).head(10)

İstanbul              813308
İstanbul (avrupa)     722512
İstanbul (anadolu)    364610
Ankara                219765
Ankara                128032
İzmir                 125185
Bursa                 122742
İstanbul              112279
İzmi̇r                 94605
Antalya                84963
Name: POSSEHIR, dtype: int64

In [29]:
print(data['POSSEHIR'].unique())

['Edi̇rne' 'İstanbul (anadolu)' 'İzmir' 'Antalya' 'İstanbul (avrupa)'
 'Samsun' 'Aydin' 'Ankara' 'İstanbul ' 'Manisa' 'Trabzon' 'Sivas' 'Si̇vas'
 'İstanbul' 'Hatay' 'Aydın' 'Adana' 'The netherlands' 'Mani̇sa'
 'Gaziantep' 'Kocaeli' 'Bursa' 'Konya' 'Mersi̇n' 'Bolu' 'Kars' 'Mersin'
 'Kırklareli' 'Denizli' 'Muğla' 'İzmi̇r' 'Antalya ' 'Tekirdağ' 'Kayseri'
 'Uşak' 'Nevşehir' 'Kayseri̇' 'Balıkesir' 'Nevşehi̇r' 'Burdur' 'Erzurum'
 'Eskişehir' 'Afyon' 'Zonguldak' 'Kastamonu' 'Malatya' 'Kahramanmaraş'
 'Mardi̇n' 'Edirne' 'Ankara ' 'Isparta' 'Kocaeli̇ ' 'Çanakkale' 'Sinop'
 'Diyarbakır' 'Kocaeli̇' 'Şırnak' 'Düzce' 'Yozgat' 'Yalova' 'Balikesi̇r'
 'Ağri' 'Kütahya' 'Düzce ' 'Teki̇rdağ' 'İzmi̇r ' 'Ordu' 'Çankiri ' 'Rize'
 'Sakarya' 'Giresun' 'Çankırı' 'Ri̇ze' 'Bartın' 'Trabzon ' 'Erzurum '
 'Eski̇şehi̇r' 'Erzincan' 'Bi̇leci̇k' 'Elazığ' 'Şanlıurfa' 'Isparta '
 'Osmaniye' 'Karaman' 'Bursa ' 'Karabük' 'Kirklareli̇' 'Eski̇şehi̇r '
 'Bilecik' 'Adıyaman' 'Çorum' 'Tokat' 'Muğla ' 'Bayburt' 'Amasya'
 'Deni̇

In [30]:
# we can change the 'İstanbul (avrupa)' and 'İstanbul (anadolu)' as 'İstanbul'

data.replace('İstanbul (anadolu)','Istanbul')
data.replace('İstanbul (avrupa)','Istanbul')
data.replace('İstanbul ','Istanbul')
data.replace('İstanbul','Istanbul')

,TOPLAMSAAT,SORUNSAYISINET,PortErrorCount,LostCarrierCount,GecenAyPortErrorSayisi,GecmisFark,Kopma,Atlak,POSSEHIR,GeneralDate
0,22,1,0,0,0,0,NaN,NaN,Edi̇rne,2020-12-01
1,22,1,0,0,0,0,NaN,NaN,Edi̇rne,2020-12-01
2,46,3,1,0,0,1,NaN,NaN,Edi̇rne,2020-12-02
3,46,3,1,0,0,1,NaN,NaN,Edi̇rne,2020-12-02
4,50,3,1,0,0,1,NaN,NaN,Edi̇rne,2020-12-03
...,...,...,...,...,...,...,...,...,...,...
4835374,12,2,1,0,0,1,NaN,NaN,Bursa,2020-12-13
4835375,0,2,0,0,0,0,NaN,NaN,İstanbul (anadolu),2020-12-13
4835376,4,2,0,0,0,0,NaN,NaN,Kocaeli̇,2020-12-13
4835377,11,1,0,0,0,0,NaN,NaN,Ankara,2020-12-13


In [31]:
# datadaki illerin dağılımınada bakalım, istanbul, ankara ve izmir domine ediyor.
data['POSSEHIR'].value_counts().sort_values(ascending=False).head(10)

İstanbul              813308
İstanbul (avrupa)     722512
İstanbul (anadolu)    364610
Ankara                219765
Ankara                128032
İzmir                 125185
Bursa                 122742
İstanbul              112279
İzmi̇r                 94605
Antalya                84963
Name: POSSEHIR, dtype: int64